In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('weight-height.csv')

In [3]:
df.columns.values

array(['Gender', 'Height', 'Weight'], dtype=object)

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(df[['Height', 'Weight']].values)
y = df['Gender'].replace('Male', 0).replace('Female', 1).values

In [5]:
# WILL UPDATE IN THE FUTURE TO USE ADAM INSTEAD

class NeuralNetwork():
    def __init__(self, X, y, number_of_nodes_in_hidden_layers):
        np.random.seed(1)

        self.X = X
        self.y = y
        self.number_of_nodes_in_hidden_layers = number_of_nodes_in_hidden_layers
        
        self.X_rows = self.X.shape[0]
        self.X_cols = self.X.shape[1]
        
        if self.X_cols >= self.number_of_nodes_in_hidden_layers:
            raise Exception('Number of nodes in hidden layers must be greater than the number of features')
        
        self.y = self.y.reshape((y.shape[0], 1))        
        
        # INITIALIZING WEIGHTS
        self.weight_0 = 2*np.random.random((self.X_cols, self.number_of_nodes_in_hidden_layers)) - 1
        self.weight_1 = 2*np.random.random((self.number_of_nodes_in_hidden_layers, self.number_of_nodes_in_hidden_layers)) - 1
        self.weight_2 = 2*np.random.random((self.number_of_nodes_in_hidden_layers, self.X_rows)) - 1
        self.weight_3 = 2*np.random.random((self.X_rows, 1)) - 1
    
    def train(self, epochs):
        for i in range(epochs):
            # # FORWARD PROPAGATION
            input_layer = self.X
            layer_1 = self.__sigmoid(np.dot(input_layer, self.weight_0))
            layer_2 = self.__sigmoid(np.dot(layer_1, self.weight_1))
            layer_3 = self.__sigmoid(np.dot(layer_2, self.weight_2))
            output_layer = self.__sigmoid(np.dot(layer_3, self.weight_3))
            
            # # BACKPROPAGATION
            output_layer_error = self.y - output_layer
            output_layer_gradient = output_layer_error*self.__sigmoid(output_layer, deriv=True)

            if (i% int(epochs/5)) == 0:
                print "Error: " + str(np.mean(np.abs(output_layer_error)))
            
            layer_3_error = output_layer_gradient.dot(self.weight_3.T)
            layer_3_gradient = layer_3_error*self.__sigmoid(layer_3, deriv=True)

            layer_2_error = layer_3_gradient.dot(self.weight_2.T)
            layer_2_gradient = layer_2_error*self.__sigmoid(layer_2, deriv=True)

            layer_1_error = layer_2_gradient.dot(self.weight_1.T)
            layer_1_gradient = layer_1_error*self.__sigmoid(layer_1, deriv=True)

            self.weight_3 = self.weight_3 + layer_3.T.dot(output_layer_gradient)
            self.weight_2 = self.weight_2 + layer_2.T.dot(layer_3_gradient)
            self.weight_1 = self.weight_1 + layer_1.T.dot(layer_2_gradient)
            self.weight_0 = self.weight_0 + input_layer.T.dot(layer_1_gradient)
    
    def predict(self, input_data):
        layer_1 = self.__sigmoid(np.dot(input_data, self.weight_0))
        layer_2 = self.__sigmoid(np.dot(layer_1, self.weight_1))
        layer_3 = self.__sigmoid(np.dot(layer_2, self.weight_2))
        output_layer = self.__sigmoid(np.dot(layer_3, self.weight_3))        

        return output_layer
    
    def __sigmoid(self, x, deriv=False):
        if (deriv==True):
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

## Test Data

In [6]:
sample_X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
sample_y = np.array([0,1,1,0])

NN = NeuralNetwork(sample_X, sample_y, 5)

NN.train(epochs=60000)
print NN.predict(np.array([0,1,1]))

Error: 0.499752648801
Error: 0.00399067501062
Error: 0.00270869413193
Error: 0.0021854241604
Error: 0.00188331273905
[ 0.99809977]


## Real Data

In [7]:
from sklearn.model_selection import train_test_split
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.2)

In [8]:
NN = NeuralNetwork(X_train, y_train, 7)
NN.train(epochs=500)

Error: 0.510481536073
Error: 0.4895
Error: 0.4895
Error: 0.4895
Error: 0.4895


In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = NN.predict(X_test)
y_test_class = y_test
y_pred_class = (y_pred > 0.45).astype(int)

In [10]:
accuracy_score(y_pred_class, y_test_class)

0.49737500000000001

In [11]:
CM = confusion_matrix(y_test_class, y_pred_class)
TP = CM[1, 1]
TN = CM[0, 0]
FP = CM[0, 1]
FN = CM[1, 0]

In [12]:
# CLASSIFICATION ACCURACY
print (TP + TN) / float(TP + TN + FP + FN)

0.497375


In [13]:
# CLASSIFICATION ERROR
print (FP + FN) / float(TP + TN + FP + FN)

0.502625


In [14]:
# Sensitivity: How sensitive is the model in predicting positive instances?
print TP / float(TP + FN)

0.0


In [15]:
# Specificity: When it's actually no, how often does it predict no?
# True Negative Rate
print TN / float(TN + FP)

1.0


In [16]:
# False Positive Rate: When it's actually no, how often does it predict yes?
print FP / float(TN + FP)

0.0


In [17]:
# Precision: When it predicts yes, how often is it correct?
print TP / float(TP + FP)

nan


/Users/JosephBautista/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in divide
  
